Liberaries imports

In [25]:
import quandl
import datetime
import cpi
from matplotlib import style

In [26]:
import os

In [27]:
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score

In [172]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

## WARNING!

`cpi.update()` is a heavy function that updates the entire CPI package and is very bloaty and unnecessary! 

**Uncomment and run if absolutely needed!**

In [50]:
# cpi.update()

In [31]:
#import API key
from SECRET_KEY import quandel_key
quandel_key

'qrznzr7vcG1HMd7ae2E7'

In [32]:
style.use('ggplot')


# ************** START OF THE RESEARCH*********************

My Course of Action:

1) Create an index for companies that could project how lyft should function. 

2) Create a function to do the these steps to a single stock
    - Import first 261 days (1 fiscal year) of stock prices for each stock
    - keep the Open, High, Low, Close, Volume, Ex-Dividend for each stock in the dataframe
    - Adjust for inflation using easy money
    - create a column to isolate the month for seasionality
    - number each instance by # of day in the year
    - Save to a .csv in the /data directory
    
3) Create a function to Graph a given dataframe

4) Create a fucntion to merge the datasets together. 

5) Normalize the full DS

6) Instaniate a linuar regression model and train it with the overall data

7) Use what Lyft stock data we have to predict to test the first year of Lyft IPO

8) Make some sexy visuals from the prediction 

9) Analize the accuracy of the model

10) Create alternative indexes and repeat this process for all of them.  


In [33]:
#Write a function to combine multiple indexes
def super_index(index_a, index_b):
    super_index = index_a
    for i in index_b:
        super_index.append(i)
    return super_index
    

In [34]:
# combiend index
# cmb_index = super_index(tech_index, transportation_index)

In [35]:
#     my_data_dict = {"years": list(),
#                     "months": list()}
    
#     for item in df["Date"]:
#         my_data_dict["years"].append(item.year)
#         my_data_dict["months"].append(item.month)

In [36]:
# Create map of DJIA and import it into the stock column
djia=pd.read_csv('data/djia.csv')
djia.columns =['Date', "DJIA"]

In [37]:
djia.head()

,Date,DJIA
0,2016-04-15,17897.46
1,2016-04-14,17926.43
2,2016-04-13,17908.28
3,2016-04-12,17721.25
4,2016-04-11,17556.41


In [38]:
def import_process_stock(stock): #Stock is a stock code string
    """Uploads the data from Quandl and runs through the various 
        processes to get the data we want adjusted for inflation. 
        once data is processed it saves to a .csv and returns data in a dataframe
        if local .csv of stock data exists it returns that data as a dataframe"""

    if os.path.isfile("data/"+stock+".csv"):
        df= pd.read_csv("data/"+stock+".csv")
        return df
    
    df = quandl.get(("WIKI/"+stock), api_key=quandel_key)
#     - keep the Open, High, Low, Close, Volume, Ex-Dividend for each stock in the dataframe
    df = df[['Open','High','Low','Close','Volume','Ex-Dividend']]
    #Import first 261 days 
#     (1 fiscal year) of stock prices for each stock
    df = df.iloc[:261]   
    #Bring the timeserise index into the datafraem
    df=df.reset_index()
    #Add day and month numbers to time serise
    df['d_number'] = range(1, len(df)+1)
#   create a column to isolate the month for seasionality
    df['Year'], df['Month'] = 0,0
    my_data_dict = {"years": list(),
                    "months": list()}
    
    for item in df["Date"]:
        my_data_dict["years"].append(int(item.year))
        my_data_dict["months"].append(int(item.month))

    df["Year"] = my_data_dict["years"]
    df["Month"] = my_data_dict["months"]

    #improt DJIA data:
    df.Date = df.Date.dt.strftime('%Y-%m-%d')
    df = df.merge(djia, how='outer', on='Date')
    df = df.iloc[:261] 

    # Adjust for inflation using Consumer Price Index
    inflated_closing_vals = list()
    inflated_high_vals = list()
    inflated_low_vals= list()
    
    for index, year in enumerate(df["Year"].astype(int)):
        inflated_closing_vals.append(round(cpi.inflate(df["Close"].iloc[index], year, to=2018), 2))
        inflated_high_vals.append(round(cpi.inflate(df["High"].iloc[index], year, to=2018), 2))
        inflated_low_vals.append(round(cpi.inflate(df["Low"].iloc[index], year, to=2018), 2))
        
    df["inflated_close"] = inflated_closing_vals
    df["inflated_high"] = inflated_high_vals
    df["inflated_low"] = inflated_low_vals

    #convert stuff to int
    df["Year"] = df["Year"].astype(int)
    df["Month"] = df["Month"].astype(int)
    df["d_number"] = df["d_number"].astype(int)
    df["Volume"] = df["Volume"].astype(int)

    # rearange table
    df=df[["Date","d_number", "Month", "Year", "Volume", "DJIA", 
           "inflated_high", "inflated_low","inflated_close"]]
#     - Save to a .csv in the /data directory
    df.to_csv("data/"+stock+".csv")


    return df

In [39]:
def process_stock_list(stock_list):
    """This function takes a list of stocks, runs the process function on them and 
        returns one large datafraame of concantonated data from all stocks"""
    #instantiate datframe
    df1 = import_process_stock(stock_list[0])
    #run a loop on all other dataframes processing and combineing them to the originonal datafame. 
    for index in range(1,len(stock_list)-1):
        new_df = import_process_stock(stock_list[index])
        df1 = pd.concat([df1, new_df])
        
    return df1
         

In [40]:
def graph_df(df):
    """"Graphs the closeing prie of a DF"""
    df["inflated_close"].plot()
    plt.legend(loc=4)
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.show()
    
    ##WHY THE FUCK IS IT PLOTING THE D_NUMBER INSTED OF DATE? WHAT THE FLYING FUCK!!

In [41]:
def open_stock_csv(stock):
    """Opens the lyft csv downlaoded from Yahoo stocks"""
    df= pd.read_csv("data/"+stock+".csv")
    # Make it look like the rest of the stocks
#     df=df.reset_index()
    #Add day and month numbers to time serise
    df['d_number'] = range(1, len(df)+1)
#   create a column to isolate the month for seasionality

    df['Year'], df['Month'] = 0,0
    my_data_dict = {"years": list(),
                    "months": list()}
    
    for item in df["Date"]:
        d_list= item.split("-")
        my_data_dict["years"].append(d_list[0])
        my_data_dict["months"].append(d_list[1])

    df["Year"] = my_data_dict["years"]
    df["Month"] = my_data_dict["months"]
    

    df["inflated_high"]= df["High"]
    df["inflated_low"]= df["Low"]
    df["inflated_close"]=df["Close"]
    #make it like the same
    df=df[["Date","d_number", "Month", "Year", "Volume", "DJIA", "inflated_close"]]
    
    
    return df

In [42]:
def svr_stock_predictor(features, df, train_size=0.75):
    """ Runs a Support Vector Machine for Regression on a stock/index. """
    
    # Train the data
    X, y = df[features], df["inflated_close"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, test_size=1 - train_size)
    
    # Fit the data
    hyper = {
        "kernel": "linear",
        "degree": 3,
        "C": 1.0,
        "gamma": 0.001
    }
    model_SVR = SVR(kernel=hyper["kernel"])
    model_SVR.fit(X_train, y_train)
    
    y_pred = model_SVR.predict(X_test)
    accuracy = model_SVR.score(X_test, y_test)
#     weights, intercept = model_SVR.coef_, model_SVR.intercept_
#     RSq, MSE = r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)
    
    ret_params = {
        "y_pred": y_pred,
        "accuracy": accuracy,
#         "weights": weights,
#         "intercept": intercept,
#         "RSq": RSq,
#         "MSE": MSE
    }
    
    return ret_params, model_SVR

In [169]:
def linear_stock_predictor(features,  df, train_size=0.75):
    """Runs a linear regression on a stock or index"""
    
    #Train the data
    X, y = df[features], df["inflated_close"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, test_size=1-train_size)
    
    #fit the data
    model_LinReg = LinearRegression()
    model_LinReg.fit(X_train, y_train)
    
    y_pred = model_LinReg.predict(X_test)
    weights, intercept = model_LinReg.coef_, model_LinReg.intercept_
    RSq, MSE = r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)
    
    ret_params = {
        "y_pred": y_pred,
        "weights": weights,
        "intercept": intercept,
        "RSq": RSq,
        "MSE": MSE
    }
    
    return ret_params, model_LinReg, y_test

In [44]:
def liner_projector(df, days):
    """This will project a stock price into the future"""
    pass
    

In [45]:
def pick_date(regressor):#Predict Price on Given Date
    date = 10
    predicted_price =regressor.predict(date)
    print(predicted_price[0][0],regressor.coef_[0][0] ,regressor.intercept_[0])

In [46]:
tester = import_process_stock(tech_index[1])

# ****WORK SECTION****

In [153]:
one_feature = ["DJIA"]
all_features = ["DJIA", "Volume", "Year", "Month"]

test_ret = linear_stock_predictor(one_feature,tester )

Run LR_model on all tech stocks and than the index to see which is most accurate

In [154]:
# 1) create an index for tech stocks that might behave like Lyft
tech_index = ['TWTR', 'GOOGL', 'FB', 'AMZN', 'SPLK', 'PYPL', 'YHOO']

#Create an index for transportation companies:
# Picked three top and bottom preforming stocks from DJTransportation index:
#  https://www.marketwatch.com/investing/index/djt

transportation_index= ['MATX', 'LUV', 'JBLU', 'CAR', 'FDX', 'ALK']

In [245]:
twtr_df=import_process_stock(tech_index[0])

avg_RSq = 0
for iteration in range(250):
    twtr_params, twtr_model, y_true =linear_stock_predictor(all_features,  twtr_df, train_size=0.75)
    avg_RSq += twtr_params['RSq']
print("{:.3f}%".format(100 * (avg_RSq / 250)))

8.793%


In [238]:
googl_df=import_process_stock(tech_index[1])

avg_RSq = 0
for iteration in range(250):
    googl_params, googl_model, y_true =linear_stock_predictor(all_features,  googl_df, train_size=0.75)
    avg_RSq += googl_params['RSq']
print("{:.3f}%".format(100 * (avg_RSq / 250)))

90.214%


In [237]:
fb_df=import_process_stock(tech_index[2])

avg_RSq = 0
for iteration in range(250):
    fb_params, fb_model, y_true =linear_stock_predictor(all_features,  fb_df, train_size=0.75)
    avg_RSq += fb_params['RSq']
print("{:.3f}%".format(100 * (avg_RSq / 250)))

31.615%


In [236]:
amzn_df=import_process_stock(tech_index[3])

avg_RSq = 0
for iteration in range(250):
    amzn_params, amzn_model, y_true =linear_stock_predictor(all_features,  amzn_df, train_size=0.75)
    avg_RSq += amzn_params['RSq']
print("{:.3f}%".format(100 * (avg_RSq / 250)))

94.263%


In [223]:
splk_df=import_process_stock(tech_index[4])

avg_RSq = 0
for iteration in range(250):
    splk_params, splk_model, y_true =linear_stock_predictor(all_features,  splk_df, train_size=0.75)
    avg_RSq += ppyl_params['RSq']
print("{:.3f}%".format(100 * (avg_RSq / 250)))

64.231%


In [222]:
pypl_df=import_process_stock(tech_index[5])

avg_RSq = 0
for iteration in range(250):
    ppyl_params, ppyl_model, y_true =linear_stock_predictor(all_features,  pypl_df, train_size=0.75)
    avg_RSq += ppyl_params['RSq']
print("{:.3f}%".format(100 * (avg_RSq / 250)))

59.335%


In [220]:
yhoo_df=import_process_stock(tech_index[6])

avg_RSq = 0
for iteration in range(250):
    yhoo_params, yhoo_model, y_true =linear_stock_predictor(all_features,  yhoo_df, train_size=0.75)
    avg_RSq += yhoo_params['RSq']
print("{:.3f}%".format(100 * (avg_RSq / 250)))

29.779%


In [163]:
# All of the tech index
tech_index_df= process_stock_list(tech_index)


tech_index_params, tech_index_model =linear_stock_predictor(all_features,  tech_index_df, train_size=0.75)
print(tech_index_params['accuracy'])

0.215662757225472


In [ ]:
linreg_ret_all, linreg_model = linear_stock_predictor(all_features,tester )

In [ ]:
linreg_ret_all

In [ ]:
print("y-line = {:.6f}X1 + {:.6f}X2 + {:.6f}X3 + {:.6f}X4 + {:.2f}".format(linreg_ret_all["weights"][0],
                                                                           linreg_ret_all["weights"][1],
                                                                           linreg_ret_all["weights"][2],
                                                                           linreg_ret_all["weights"][3], 
                                                                           linreg_ret_all["intercept"]))

print("\nX1: {}\nX2: {}\nX3: {}\nX4: {}\n".format(all_features[0],
                                                all_features[1],
                                                all_features[2],
                                                all_features[3]))

lookup_year, lookup_month = 2019, 6
#            DJIA      Vol         Year         Month
new_data = [[17897.46, 12345600.0, lookup_year, lookup_month],
            [18550.62, 21156200.0, lookup_year, lookup_month]]
linreg_model.predict(new_data)

In [ ]:
sns.heatmap(ester.corr())

In [ ]:
df_lyft= open_stock_csv('LYFT')

In [ ]:
tech_index

In [ ]:
big_DFF = process_stock_list(tech_index)

In [ ]:
big_DFF.to_csv('data/tech_index.csv')